### Setup

In [1]:
import pandas as pd
import os
import csv

##### Considerations

Orginal Dataset from Kaggle: https://www.kaggle.com/sobhanmoosavi/us-accidents

Since raw file weighted ~1.5 GB, we had to clean up the data (filter Start_Time only year 2019) and compress the CSV file to allow the csv to be pushed within GitHub's threshold.

###### Compression Information

compression_opts = dict(method='zip', archive_name='us_accidents_2019_zipped.csv')

us_accidents.to_csv('us_accidents_2019_zipped.csv', index=False, compression=compression_opts)

In [2]:
file_path = './us_accidents_2019_zipped.csv'
us_accidents = pd.read_csv(file_path, compression='zip')
us_accidents

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-879261,MapQuest,406.0,3,2019-12-26 17:09:52,2020-01-10 11:00:00,40.884850,-95.800362,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
1,A-879262,MapQuest,406.0,3,2019-12-26 17:10:32,2020-01-10 11:00:00,40.889641,-95.798630,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
2,A-982553,MapQuest,201.0,2,2019-12-25 09:49:34,2019-12-25 11:03:57,41.848064,-71.062386,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
3,A-982554,MapQuest,201.0,2,2019-12-25 12:52:38,2019-12-25 13:37:03,42.660690,-71.192589,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-982555,MapQuest,201.0,2,2019-12-25 20:43:26,2019-12-25 21:58:45,42.157021,-71.069061,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950969,A-4239402,Bing,NaN,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,33.99888,-117.37094,...,False,False,False,False,False,False,Day,Day,Day,Day
950970,A-4239403,Bing,NaN,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,32.76555,-117.15363,...,False,False,False,False,False,False,Day,Day,Day,Day
950971,A-4239404,Bing,NaN,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,33.77740,-117.85727,...,False,False,False,False,False,False,Day,Day,Day,Day
950972,A-4239405,Bing,NaN,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,33.98311,-118.39565,...,False,False,False,False,False,False,Day,Day,Day,Day


In [3]:
us_accidents["State"]

0         IA
1         IA
2         MA
3         MA
4         MA
          ..
950969    CA
950970    CA
950971    CA
950972    CA
950973    CA
Name: State, Length: 950974, dtype: object

### US Census

In [4]:
# Read csv file
census_file_path = './ACSST1Y2019.S0804-2021-04-06T021956.csv'

metadata = pd.read_csv(census_file_path)

In [5]:
# Read states and each row with totals
states = []
rows = []

csvpath = os.path.join(census_file_path)

with open(csvpath, encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    sv_header = next(csvreader)
    
    # Read each row of data after the header
    is_total = False
    for row in csvreader:
        if not row[0].startswith('\xa0'):
            states.append(row[0])
            
        if is_total:
            rows.append(row)
            is_total = False
            
        if row[0].startswith('\xa0\xa0\xa0\xa0Total'):
            is_total = True

In [6]:
# Convert state list to dataframe
states_df = pd.DataFrame(states, columns=['State'])
states_df['State'] = states_df['State'].str.upper()

# Convert row list to dataframe
rows_df = pd.DataFrame(rows, columns=metadata.columns)
rows_df["State"] = states_df["State"]

In [7]:
# Extract only our selected columns and set State as index
sel_col_df = rows_df[[
    'State',
    'Workers 16 years and over',
    'Workers 16 years and over!!SEX!!Male',
    'Workers 16 years and over!!SEX!!Female',
    'Workers 16 years and over!!AGE!!16 to 19 years',
    'Workers 16 years and over!!AGE!!20 to 24 years',
    'Workers 16 years and over!!AGE!!25 to 44 years',
    'Workers 16 years and over!!AGE!!45 to 54 years',
    'Workers 16 years and over!!AGE!!55 to 59 years',
    'Workers 16 years and over!!AGE!!60 years and over'
    
]]
sel_col_df.set_index('State', inplace=True)

In [8]:
# Rename selected columns
sel_col_df.rename(columns={
    "Workers 16 years and over!!SEX!!Male": "Male",
    "Workers 16 years and over!!SEX!!Female" : "Female",
    "Workers 16 years and over!!AGE!!16 to 19 years": "Age_16_19",
    "Workers 16 years and over!!AGE!!20 to 24 years": "Age_20_24",
    "Workers 16 years and over!!AGE!!25 to 44 years": "Age_25_44",
    "Workers 16 years and over!!AGE!!45 to 54 years": "Age_45_54",
    "Workers 16 years and over!!AGE!!55 to 59 years": "Age_55_59",
    "Workers 16 years and over!!AGE!!60 years and over": "Age_60_",
    "Workers 16 years and over": "Total"
}, inplace=True)

C:\Users\Wayssen\anaconda3\envs\pythonData\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
# Clean and convert to float type the total column
total_df = sel_col_df[["Total"]]
total_df = total_df.iloc[:,:].apply(lambda x: x.str.replace(',', '').astype(float) , axis=1 )

In [10]:
# Clean and convert to float type all percentage columns
content_df = sel_col_df.iloc[:,1:]
content_df = content_df.iloc[:,:].apply(lambda x: x.str.replace('%', '').astype(float)/100 , axis=1 )

In [11]:
# Merge cleaned index with cleaned percentage columns
merged_df = total_df.merge(content_df, on="State")

In [12]:
# Use merge to multipy each percentage column with total column
merged_qty_df = merged_df.multiply(merged_df["Total"], axis="index")
merged_qty_df.drop(columns=["Total"], inplace=True)

# Restore original total values
merged_qty_df["Total"] = total_df["Total"]

# Reorder columns
merged_qty_df = merged_qty_df[["Total","Male","Female","Age_16_19","Age_20_24","Age_25_44","Age_45_54","Age_55_59", "Age_60_"]]

# Convert all columns to int type
merged_qty_df = merged_qty_df.iloc[:,:].apply(lambda x: round(x, 0).astype(int)  , axis=1 )

In [13]:
# Extract us state codes
code_states = pd.read_html("https://www.ssa.gov/international/coc-docs/states.html")
code_states_df = code_states[0].rename(columns={0: "State", 1: "Code"})

# Merge extracted codes
merged_qty_df = merged_qty_df.merge(code_states_df, on="State")

# Rename merged columns
merged_qty_df["State_Name"] = merged_qty_df["State"]
merged_qty_df["State"] = merged_qty_df["Code"]
merged_qty_df.drop(columns=["Code"], inplace=True)

In [14]:
# Save and show cleaned U.S. Census Data
merged_qty_df.to_csv("us_census_cleaned.csv")

merged_qty_df.head(2)

,State,Total,Male,Female,Age_16_19,Age_20_24,Age_25_44,Age_45_54,Age_55_59,Age_60_,State_Name
0,AL,2093631,1097063,996568,81652,198895,910729,429194,196801,276359,ALABAMA
1,AK,358536,202931,155605,13624,33702,163134,69556,33702,44817,ALASKA
